# Clash of Clans CWL
## Data pipeline build

* Your Name: Nolan Clark
* Name of API: Clash of Clans API
* Link to main page of API documentation: https://developer.clashofclans.com/#/documentation

In [1]:
# Imports go here
import requests
import pandas as pd
import numpy as np
import MyKeys

import json
import datetime

import CWL_Functions as cf
from collections import Counter
from collections import defaultdict

import time

# API Token used with the documented Authorization header. 
# The headers are the necessary params of each endpoint
MyToken = MyKeys.MyToken
headers = {'authorization': 'Bearer '+(MyToken), 'Accept': 'application/json'}
base = 'https://api.clashofclans.com/v1/'

# Automate pointers for the pipeline -- avoid erasing previous month
month=datetime.datetime.now().strftime('%B')

In [2]:
start_all= time.time()
LeagueBins = {'Bronze League III': [],
 'Bronze League II': [],
 'Bronze League I': [],
 'Silver League III': [],
 'Silver League II': [],
 'Silver League I': [],
 'Gold League III': [],
 'Gold League II': [],
 'Gold League I': [],
 'Crystal League III': [],
 'Crystal League II': [],
 'Crystal League I': [],
 'Master League III': [],
 'Master League II': [],
 'Master League I': [],
 'Champion League III': [],
 'Champion League II': [],
 'Champion League I': []}

In [3]:
tag_bin = {'Bronze League III': [],
 'Bronze League II': [],
 'Bronze League I': [],
 'Silver League III': [],
 'Silver League II': [],
 'Silver League I': [],
 'Gold League III': [],
 'Gold League II': [],
 'Gold League I': [],
 'Crystal League III': [],
 'Crystal League II': [],
 'Crystal League I': [],
 'Master League III': [],
 'Master League II': [],
 'Master League I': [],
 'Champion League III': [],
 'Champion League II': [],
 'Champion League I': []}

In [4]:
def meltLeague(ClanTag):
    url='https://api.clashofclans.com/v1/clans/%23'+ClanTag
    response = requests.get(url, headers=headers, timeout=30)
    #print(response.status_code, response.reason)
    league= response.json()['warLeague']['name']
    
    
    return league

In [5]:
def grabTagsBig(ClanTag):
    collect = []
    
    url='https://api.clashofclans.com/v1/clans/%23'+ClanTag+'/currentwar/leaguegroup'
    response = requests.get(url, headers=headers, timeout=30)
    #print(response.status_code, response.reason)
    
    
    # range of 7 so that only CWL with all rounds will be pulled
    for i in range(7):
        collect.extend(response.json().get('rounds')[i].get('warTags'))
    
    collDict = collect
    
    return collDict

In [6]:
def search_clans_v2(**payload):
    
    
    url='https://api.clashofclans.com/v1/clans?'
    response = requests.get(url, headers=headers, params = payload, timeout=30)
    print(response.status_code, response.reason)
    
    top_clans=[i['tag'].strip('#') for i in response.json()['items']]
    
    #top_clans=response.json()
    
    try:
        after = response.json()['paging'].get('cursors')['after']
    except:
        after = False
    
    return top_clans, after

In [7]:
start = time.time()
bucket = []
clanPointInts = [20000,30000,40000,50000]

for points in clanPointInts:
    params = {'limit':'100','minClanPoints':points}
    after_page = ''
    while after_page != False:
        all_tags = search_clans_v2(**params)
        bucket.extend(all_tags[0])
        after_page = all_tags[1]
        params['after'] = after_page
        
end = time.time()
print(end - start)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
8.66434097290039


In [8]:
len(pd.unique(bucket))

3169

In [9]:
len(bucket)

3328

In [10]:
top_clans = pd.unique(bucket)

In [11]:
start = time.time()
inWar_bin = []
for i in top_clans:
    j = cf.status15(i)
    if j != False:
        inWar_bin.append(i)
end = time.time()
print(end - start)

1329.420260667801


In [12]:
start = time.time()
for clan in inWar_bin:
    tag_bin[meltLeague(clan)].append(clan)
end = time.time()
print(end - start)

317.5609300136566


# Break This Step and split into two
grab clan tags in beginning, write to JSON, then move forward around the end

In [13]:
len(inWar_bin)

1259

In [14]:
{key: (len(tag_bin[key])) for key in  tag_bin.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 1,
 'Silver League II': 4,
 'Silver League I': 8,
 'Gold League III': 16,
 'Gold League II': 57,
 'Gold League I': 69,
 'Crystal League III': 124,
 'Crystal League II': 173,
 'Crystal League I': 235,
 'Master League III': 174,
 'Master League II': 125,
 'Master League I': 85,
 'Champion League III': 99,
 'Champion League II': 60,
 'Champion League I': 29}

In [15]:
{key: (len(pd.unique(tag_bin[key]))) for key in  tag_bin.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 1,
 'Silver League II': 4,
 'Silver League I': 8,
 'Gold League III': 16,
 'Gold League II': 57,
 'Gold League I': 69,
 'Crystal League III': 124,
 'Crystal League II': 173,
 'Crystal League I': 235,
 'Master League III': 174,
 'Master League II': 125,
 'Master League I': 85,
 'Champion League III': 99,
 'Champion League II': 60,
 'Champion League I': 29}

In [16]:
# LARGE GRAB

leagueNames=list(tag_bin.keys())

for league in leagueNames:
    print(league)
    for tag in tag_bin[league]:
        try:
            rounds = grabTagsBig(tag)
            LeagueBins[league].extend(rounds)
        except (TypeError, IndexError):
            pass

Bronze League III
Bronze League II
Bronze League I
Silver League III
Silver League II
Silver League I
Gold League III
Gold League II
Gold League I
Crystal League III
Crystal League II
Crystal League I
Master League III
Master League II
Master League I
Champion League III
Champion League II
Champion League I


# Diagnostics 

In [17]:
end_all = time.time()
print(end_all - start_all)

1961.7520570755005


In [18]:
{key: (len(LeagueBins[key])) for key in  LeagueBins.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 28,
 'Silver League II': 112,
 'Silver League I': 224,
 'Gold League III': 448,
 'Gold League II': 1596,
 'Gold League I': 1932,
 'Crystal League III': 3472,
 'Crystal League II': 4816,
 'Crystal League I': 6580,
 'Master League III': 4872,
 'Master League II': 3500,
 'Master League I': 2380,
 'Champion League III': 2772,
 'Champion League II': 1652,
 'Champion League I': 812}

In [19]:
{key: (len(pd.unique(LeagueBins[key]))) for key in  LeagueBins.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 28,
 'Silver League II': 112,
 'Silver League I': 224,
 'Gold League III': 448,
 'Gold League II': 1593,
 'Gold League I': 1904,
 'Crystal League III': 3472,
 'Crystal League II': 4788,
 'Crystal League I': 6384,
 'Master League III': 4816,
 'Master League II': 3276,
 'Master League I': 2268,
 'Champion League III': 2520,
 'Champion League II': 1484,
 'Champion League I': 616}

In [20]:
cwlRoundsBins={key: list(pd.unique(LeagueBins[key])) for key in  LeagueBins.keys()}

In [21]:
# write the updated file
with open(input('File name for the Pipeline to be saved as: '),"w") as write_file:
    json.dump(cwlRoundsBins, write_file)

File name for the Pipeline to be saved as: PipelineRunAug.json


In [22]:
# write the updated file
with open(f"PipelineRun{month}.json","w") as write_file:
    json.dump(cwlRoundsBins, write_file)